In [ ]:
# Connect to Drive
from google.colab import drive
drive.mount("/content/gdrive")

# Verify the connection
!pwd
# !ls -lht gdrive/'Shared drives'/'2020_FDLUSA_Earth Science_Knowledge Discovery Framework'/Code/Datasets/UCMerced_LandUse/Images

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content


In [ ]:
# Library Imports
import os
import time
import math
import pickle
import random
import tensorflow
import numpy as np
from numpy.linalg import norm
from tqdm import tqdm, tqdm_notebook
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D

# # Custom Imports
# !pip install import_ipynb
# import import_ipynb
# import utils

In [ ]:
os.chdir("/content/gdrive/Shared drives/2020_FDLUSA_Earth Science_Knowledge Discovery Framework/Code/Week5_Code/")

In [ ]:
# Function to choose the predefined Model architecture
def model_picker(name):
    if (name == 'vgg16'):
        model = VGG16(weights='imagenet',
                      include_top=False,
                      input_shape=(224, 224, 3),
                      pooling='max')
    elif (name == 'vgg19'):
        model = VGG19(weights='imagenet',
                      include_top=False,
                      input_shape=(224, 224, 3),
                      pooling='max')
    elif (name == 'mobilenet'):
        model = MobileNet(weights='imagenet',
                          include_top=False,
                          input_shape=(224, 224, 3),
                          pooling='max',
                          depth_multiplier=1,
                          alpha=1)
    elif (name == 'inception'):
        model = InceptionV3(weights='imagenet',
                            include_top=False,
                            input_shape=(224, 224, 3),
                            pooling='max')
    elif (name == 'resnet'):
        model = ResNet50(weights='imagenet',
                         include_top=False,
                         input_shape=(224, 224, 3),
                        pooling='max')
    elif (name == 'xception'):
        model = Xception(weights='imagenet',
                         include_top=False,
                         input_shape=(224, 224, 3),
                         pooling='max')
    else:
        print("Specified model not available")
    return model

# Get the pretrained model
model_architecture = 'resnet'
model = model_picker(model_architecture)

# # To Get custom model
# modelPath='Models/Colorization_CustomCNN_epoch30_ucmerced'
# model=load_model(modelPath)

model.summary()

94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
# Function to featurize the input
def extract_features(img, model):
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    features = model.predict(preprocessed_img)
    flattened_features = features.flatten()
    normalized_features = flattened_features / norm(flattened_features)
    return normalized_features

In [ ]:
# Function to get the list of input file names
extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG', '.tif']
def get_file_list(root_dir):
    file_list = []
    for root, directories, filenames in os.walk(root_dir):
        for filename in filenames:
            # print("filename:", filename)
            if any(ext in filename for ext in extensions):
                file_list.append(os.path.join(root, filename))
    return file_list

# Dataset location
root_dir = '/content/gdrive/Shared drives/2020_FDLUSA_Earth Science_Knowledge Discovery Framework/Code/Datasets/UCMerced_LandUse/Images/Splits_3/test/'
filenames = sorted(get_file_list(root_dir))

print(filenames)

# Featurize all input images
feature_list = []
for i in tqdm_notebook(range(len(filenames))):
    input_shape = (224, 224, 3)
    img = image.load_img(filenames[i], target_size=(input_shape[0], input_shape[1]))    

    feature_list.append(extract_features(img, model))

In [ ]:
# Save the features and the filelist order for later use.
pickle.dump(feature_list, file=open(("../Features/customcolorizer_merced_combined_features.pkl"), mode = 'wb'))
pickle.dump(filenames, file = open(("../Features/customcolorizer_merced_combined_filenames.pkl"), mode = 'wb'))